In [ ]:
#!/usr/bin/env python
# coding: utf-8

"""
AGV 메인 시스템 - JSON 송수신 기능 포함 (item_idx 지원)
"""

import sys
import os
import time
import threading
from datetime import datetime

# 로컬 모듈 임포트
sys.path.append(os.path.dirname(os.path.abspath(__file__)))
sys.path.append(os.path.join(os.path.dirname(os.path.abspath(__file__)), 'control'))
from config import *
from mqtt_manager import MQTTManager
from road_following import RoadFollowing
from area_detecting import AreaDetection

# 하드웨어 라이브러리
import torchvision
import torch
import torchvision.transforms as transforms
from jetbot import Robot, Camera

class AGVSystem:
    def __init__(self):
        # 하드웨어
        self.robot = None
        self.camera = None
        
        # AI 모델
        self.model = None
        self.mean = None
        self.std = None
        
        # 시스템 컴포넌트
        self.mqtt_manager = None
        self.road_following = None
        self.area_detection = None
        
        # 상태
        self.current_task = None
        self.is_running = False
        
    def initialize(self):
        """시스템 초기화"""
        # 하드웨어 초기화
        self.robot = Robot()
        self.camera = Camera()
        
        # AI 모델 로드
        self.model = torchvision.models.resnet18(pretrained=False)
        self.model.fc = torch.nn.Linear(512, 2)
        self.model.load_state_dict(torch.load(MODEL_PATH))
        
        device = torch.device('cuda')
        self.model = self.model.to(device).eval().half()
        
        self.mean = torch.Tensor(IMAGENET_MEAN).cuda().half()
        self.std = torch.Tensor(IMAGENET_STD).cuda().half()
        
        # 시스템 컴포넌트 초기화 (카메라를 MQTT 매니저에 전달)
        self.mqtt_manager = MQTTManager(
            command_callback=self._handle_command,
            camera=self.camera
        )
        self.road_following = RoadFollowing(self.camera, self.robot, self.model, self.mean, self.std)
        self.area_detection = AreaDetection(self.camera, road_following_controller=self.road_following)
        self.area_detection.set_callbacks(task_complete_callback=self._on_task_completed)
        
        # 로드 팔로잉 컨트롤러를 영역 탐지에 연결
        self.area_detection.set_road_following_controller(self.road_following)
        
        # 스레드 시작
        self.road_following.start()
        self.area_detection.start()
        
        print("AGV 시스템 초기화 완료")
        
    def _handle_command(self, command_data):
        """
        명령 처리 - item_idx 필드 포함
        수신 형식: {
            "timedata": "2025-05-22 14:30:15",
            "start": "orange",
            "end": "blue", 
            "delays": 5,
            "item_idx": 3
        }
        """
        print(f"작업 명령 처리 시작: {command_data}")
        
        # 기존 작업이 실행 중이면 무시
        if self.is_running:
            print("기존 작업 실행 중 - 새 명령 무시")
            return
            
        # 명령 데이터 저장
        self.current_task = {
            "timedata": command_data.get("timedata"),
            "start": command_data.get("start"),
            "end": command_data.get("end"),
            "delays": command_data.get("delays", 0),
            "item_idx": command_data.get("item_idx", 0)
        }
        
        delays = self.current_task["delays"]
        
        # 지연 시간 후 작업 시작
        if delays > 0:
            print(f"{delays}초 후 작업 시작 예정")
            threading.Timer(delays, self._start_task).start()
        else:
            self._start_task()
    
    def _start_task(self):
        """작업 시작"""
        if not self.current_task:
            print("작업 데이터 없음")
            return
            
        print(f"작업 시작: {self.current_task['start']} → {self.current_task['end']}")
        print(f"물건 인덱스: {self.current_task['item_idx']}")
        
        start = self.current_task["start"]
        end = self.current_task["end"]
        item_idx = self.current_task["item_idx"]
        
        # 목표 영역 설정
        self.area_detection.set_target_areas(start, end)
        
        # 물건 인덱스 설정 (필요시 사용)
        self.area_detection.set_item_index(item_idx)
        
        # 라인팔로잉과 영역탐지 시작
        self.road_following.start_following()
        self.area_detection.start_detection()
        
        # 센서 데이터 송신 시작 (0.5초마다)
        self.mqtt_manager.start_sensing_transmission()
        
        self.is_running = True
        print("작업 실행 중 - 센서 데이터 송신 시작")
    
    def _on_task_completed(self):
        """작업 완료 처리"""
        print("작업 완료 처리 시작")
        
        # 라인팔로잉 정지
        self.road_following.stop_following()
        
        # 작업 완료 상태 설정
        self.mqtt_manager.set_task_finished()
        
        # 잠시 대기 후 센서 데이터 송신 정지
        threading.Timer(2.0, self._finalize_task).start()
    
    def _finalize_task(self):
        """작업 완전 종료"""
        # 센서 데이터 송신 정지
        self.mqtt_manager.stop_sensing_transmission()
        
        # 상태 초기화
        self.current_task = None
        self.is_running = False
        
        print("작업 완료 - 대기 상태로 복귀")
    
    def run(self):
        """메인 실행 루프"""
        self.initialize()
        
        # MQTT 연결
        print("MQTT 브로커 연결 시도 중...")
        while not self.mqtt_manager.connect():
            print("MQTT 연결 실패 - 1초 후 재시도")
            time.sleep(1)
        
        print("MQTT 연결 성공 - 명령 대기 중")
        
        try:
            # 명령 대기 루프
            while True:
                time.sleep(0.1)
        except KeyboardInterrupt:
            print("사용자 중단 요청")
        finally:
            self.shutdown()
    
    def shutdown(self):
        """시스템 종료"""
        print("AGV 시스템 종료 중...")
        
        if self.road_following:
            self.road_following.stop()
        if self.area_detection:
            self.area_detection.stop()
        if self.mqtt_manager:
            self.mqtt_manager.disconnect()
        if self.robot:
            self.robot.stop()
        if self.camera:
            self.camera.stop()
            
        print("AGV 시스템 종료 완료")

def main():
    agv_system = AGVSystem()
    agv_system.run()

if __name__ == "__main__":
    main()